In [24]:
import requests
import os

API_KEY = os.getenv("API_KEY")

SPORT = 'soccer_epl' 
REGIONS = 'uk' 
MARKETS = 'h2h' 
ODDS_FORMAT = 'decimal' 
DATE_FORMAT = 'iso' 
BOOKIES = "williamhill,paddypower,unibet_uk,skybet,marathonbet"


In [25]:
odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
        'bookmakers': BOOKIES
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 489
Used requests 11


In [39]:
import pandas as pd
from collections import defaultdict

df = pd.json_normalize(
    odds_json,
    record_path=['bookmakers','markets','outcomes'],
    meta=['id','sport_key','sport_title','commence_time','home_team','away_team',
            ['bookmakers','title'], ['bookmakers','last_update'],
            ['bookmakers','markets','last_update']],
    errors='ignore'
)

df = (df.rename(columns={
        'id':'event_id',
        'bookmakers.title':'bookmaker',
        'bookmakers.last_update':'bookmaker_last_update',
        'bookmakers.markets.last_update':'market_last_update',
        'name':'outcome_name',
        'price':'odds_decimal'
    })
    [['event_id','commence_time','home_team','away_team',
        'bookmaker','bookmaker_last_update',
        'outcome_name','odds_decimal']]
)

print(df.head())
# def role_from(outcome, home, away):
#     o = str(outcome).strip().casefold()
#     if o == str(home).strip().casefold(): return "home"
#     if o == str(away).strip().casefold(): return "away"
#     if o == "draw": return "draw"
#     return None

# events = defaultdict(dict)
# for eid, g in df.groupby(id): 
#     home = g["home_team"].iat[0]
#     away = g["away_team"].iat[0]
#     dc = {
#         "commence_time": g["commence_time"].iat[0],
#         "home_team": home,
#         "away_team": away,
#     }    
#     teams = defaultdict(list)
#     draw = False 
    
#     for _, r in g.iterrows():
#         role = role_from(r["outcome_name"], home, away)
#         if role is None:
#             continue
#         if role == "draw":
#             has_draw = True
#             legs.setdefault("draw", [])
#         legs[role].append({"odd": float(r["odds_decimal"]), "book": r["bookmaker"]})

#     events[eid] = {"meta": meta, "legs": legs if has_draw else {"home": legs["home"], "away": legs["away"]}}
    


                           event_id         commence_time          home_team  \
0  354123128ddb07b54e61c8f0953029df  2025-10-25T16:30:00Z  Manchester United   
1  354123128ddb07b54e61c8f0953029df  2025-10-25T16:30:00Z  Manchester United   
2  354123128ddb07b54e61c8f0953029df  2025-10-25T16:30:00Z  Manchester United   
3  354123128ddb07b54e61c8f0953029df  2025-10-25T16:30:00Z  Manchester United   
4  354123128ddb07b54e61c8f0953029df  2025-10-25T16:30:00Z  Manchester United   

                  away_team    bookmaker bookmaker_last_update  \
0  Brighton and Hove Albion  Paddy Power  2025-10-25T17:34:14Z   
1  Brighton and Hove Albion  Paddy Power  2025-10-25T17:34:14Z   
2  Brighton and Hove Albion  Paddy Power  2025-10-25T17:34:14Z   
3  Brighton and Hove Albion  Unibet (UK)  2025-10-25T17:35:01Z   
4  Brighton and Hove Albion  Unibet (UK)  2025-10-25T17:35:01Z   

               outcome_name  odds_decimal  
0  Brighton and Hove Albion          26.0  
1         Manchester United       